In [1]:
import sqlite3
conn = sqlite3.connect('./test.db')

### 테이블 생성

In [3]:
cur = conn.cursor()
cur.execute(""" CREATE TABLE IF NOT EXISTS Bears (
                    back_no INT NOT NULL,
                    name TEXT,
                    position TEXT,
                    PRIMARY KEY(back_no)
                )""")

### 테이블 구조/이름 변경

In [22]:
# cur.execute('ALTER TABLE Bears ADD COLUMN birth INTEGER')
# cur.execute('ALTER TABLE Bears RENAME TO Eagles')
cur.execute("ALTER TABLE 'Eagles' DROP 'birth';")

""" SQLite에서는 drop COLUMN을 지원하지 않는다고?
=> 새 테이블을 만들고 원래 테이블을 지운 다음 새 테이블 이름을 바꾸라는데...
 """

OperationalError: near "DROP": syntax error

In [28]:
# cur.execute("Drop table Eagles")

In [29]:
cur = conn.cursor()
cur.execute(""" CREATE TABLE IF NOT EXISTS Eagles (
                    back_no INT NOT NULL,
                    name TEXT,
                    position TEXT,
                    PRIMARY KEY(back_no)
                )""")

### 데이터 삽입

In [30]:
cur = conn.cursor()
cur.execute("""INSERT INTO Eagles VALUES(1,'하주석','내야수'),
                    (57,'정우람','투수'),
                    (8,'정근우','내야수')""")

In [33]:
# 파라미터는 튜플로 사용
back_no = 50
name = '이성열'
position = '외야수'
cur = conn.cursor()
sql = 'INSERT INTO Eagles VALUES(?,?,?)'
cur.execute(sql, (back_no,name,position))

IntegrityError: UNIQUE constraint failed: Eagles.back_no

In [34]:
conn.commit()

In [35]:
# 2차원 튜플
players = (
    (22, '이태양', '투수'),
    (13, '최재훈', '포수')
)
cur = conn.cursor()
sql = 'INSERT INTO Eagles VALUES (?,?,?)'
cur.executemany(sql, players)
conn.commit()

In [37]:
cur.execute('SELECT * FROM Eagles ORDER BY name DESC LIMIT 1')
row = cur.fetchone()
row

(1, '하주석', '내야수')

In [38]:
# 파라미터가 1개일 때에도 튜플로 넣어줘야 함
cur = conn.cursor()
back_no = 50
cur.execute('SELECT * FROM Eagles WHERE back_no = ?', (back_no,))
player = cur.fetchone()
player

(50, '이성열', '외야수')

### 연습 문제

- 1번

In [58]:
cur = conn.cursor()
cur.execute(""" CREATE TABLE Pitcher_stats (
    name text,
    ERA Real,
    IP INTEGER,
    SO INTEGER,
    foreign key(name)
    references Eagles(name)
)""")
conn.commit()

In [59]:
cur.execute(""" INSERT INTO Pitcher_stats VALUES('하주석', 0.58 , 2300 , 2700),
                    ('정우람', 0.65 , 2280, 3000 ),
                    ('정근우', 0.71 , 2650, 2900 ), 
                    ('이성열', 0.55 , 2710, 2700 ), 
                    ('이태양', 0.59 , 2360, 3100 ), 
                    ('최재훈', 0.63 , 2650, 2600 ) 
                    """)

In [49]:
conn.commit()

In [50]:
cur.execute(""" select * from Eagles
    left join Pitcher_stats
    on Pitcher_stats.name = Eagles.name
""")
rows = cur.fetchall()
for row in rows:
    print(row)

(1, '하주석', '내야수', '하주석', 0.58, 2300, 2700)
(57, '정우람', '투수', '정우람', 0.65, 2280, 3000)
(8, '정근우', '내야수', '정근우', 0.71, 2650, 2900)
(50, '이성열', '외야수', '이성열', 0.55, 2710, 2700)
(22, '이태양', '투수', '이태양', 0.59, 2360, 3100)
(13, '최재훈', '포수', '최재훈', 0.63, 2650, 2600)


- 2-1번

In [78]:
cur.execute(""" create table if not exists band (
    id INT NOT NULL,
    group_name TEXT,
    population INT,
    debut DATE,
    agency TEXT,
    PRIMARY KEY(id)
)
""")

In [79]:
cur.execute(""" INSERT INTO band VALUES(1, '방탄소년단', 7 , "2016-09-10" , '빅히트'),
                    (2 ,'소녀시대', 9 , "2009-04-14", 'SM' ),
                    (3 ,'트와이스', 9 , "2016-05-01", 'JYP' ), 
                    (4 ,'블랙핑크', 5 , "2015-12-26", 'YG' ), 
                    (5 ,'레드벨벳', 5 , "2015-11-31", 'SM' ), 
                    (6 ,'ITZY', 5 , "2020-11-12", 'JYP' ) 
""")

In [80]:
conn.commit()

- 2-2번

In [89]:
cur.execute(""" create table song (
    song_id INT NOT NULL,
    song_name TEXT,
    group_id INT,
    release DATE,
    composer TEXT,
    lyrics TEXT,
    PRIMARY KEY(song_id),
    foreign key(group_id)
    references Eagles(name)
)
""")

OperationalError: table song already exists

In [88]:
cur.execute(""" INSERT INTO song VALUES(10001, '다이너마이트', 1 , "2020-03-04" , "방시혁", "Cause I-I-I'm in the stars tonight"),
        (10002 ,'Gee', 2 , "2009-04-14", "TED", '너무너무 멋져 눈이 눈이 부셔' ),
        (10003 ,'우아하게', 3 , "2016-09-10", "신사동호랭이", '매일 가만 안 두죠'), 
        (10004 ,' How You Like That', 4 , "2018-08-14", "신당동떡볶이", '보란 듯이 무너졌어' ), 
        (10005 ,'러시안룰렛', 5 , "2019-02-11", "이수만", '날카로운 secret 둘러싼' ), 
        (10006 ,'ICY', 6 , "2019-05-09", "박진영", '차갑게 보여도 어떡해 cool한 나니까' ) 
""")

IntegrityError: UNIQUE constraint failed: song.song_id

In [90]:
conn.commit()

- 2-3번

In [93]:
cur.execute(""" select band.group_name, band.population, band.debut, song.song_name, song.release from band
inner join song
where song.group_id = band.id
""")
rows = cur.fetchall()
for row in rows:
    print(row)

('방탄소년단', 7, '2016-09-10', '다이너마이트', '2020-03-04')
('소녀시대', 9, '2009-04-14', 'Gee', '2009-04-14')
('트와이스', 9, '2016-05-01', '우아하게', '2016-09-10')
('블랙핑크', 5, '2015-12-26', ' How You Like That', '2018-08-14')
('레드벨벳', 5, '2015-11-31', '러시안룰렛', '2019-02-11')
('ITZY', 5, '2020-11-12', 'ICY', '2019-05-09')


- 3번

In [94]:
cur.execute(""" create table users (
    name text,
    pwd varchar(400)
)
""")

In [97]:
cur.execute(""" INSERT INTO users VALUES ('최원태',1234),
('송명수',1234),
('박은채',1234),
('김동우',2345),
('이예지',1234),
('배성민',4567)
""")
conn.commit()

In [109]:
name, pwd = input('이름 패스워드 입력:').split(' ')

In [111]:
info = (name, pwd)

class WrongInputError(Exception):
    def __init__(self):
        super().__init__('잘못된 입력입니다.')

try:
    cur.execute(""" select * from users where name = ? and pwd = ?
    """,info)
    if cur.fetchone() == True:
        print(cur.fetchone())
    else:
        raise WrongInputError
    
except Exception as e:
    print(e)

잘못된 입력입니다.
